In [1]:
from dotenv import load_dotenv
import os
import sys
sys.path.append('/home/r42/caseWill')

from tqdm import tqdm
from utils.connections import duckdb_load_postgres

In [2]:
load_dotenv()
PATH_RAW = os.getenv('MINIO_RAW')
db = duckdb_load_postgres()

In [7]:
create_raw_costumer = """
    CREATE TABLE IF NOT EXISTS raw.customer (
        entry_date          DATE        NULL,
        surrogate_key       INTEGER     NULL,
        full_name           VARCHAR     NULL,
        birth_date          DATE        NULL,
        uf_name             VARCHAR     NULL,
        uf                  VARCHAR     NULL,
        street_name         VARCHAR     NULL
    );
    
    WITH landing AS (
        SELECT 
            * 
        FROM read_csv_auto('{PATH_RAW}/customer.csv')
    )
    INSERT INTO raw.customer (
        entry_date, 
        surrogate_key, 
        full_name, 
        birth_date, 
        uf_name, 
        uf, 
        street_name
    )
    SELECT * FROM landing
"""

create_raw_core_account = """
    CREATE TABLE IF NOT EXISTS raw.core_account (
        dt_transaction          DATE        NULL,
        dt_month                INTEGER     NULL,
        surrogate_key           INTEGER     NULL,
        cd_seqlan               INTEGER     NULL,
        ds_transaction_type     VARCHAR     NULL,
        vl_transaction          DOUBLE      NULL,
        id_transaction          VARCHAR     NULL
    );
    
    WITH df AS (
        SELECT 
            *
        FROM read_csv_auto('{PATH_RAW}/core_account.csv')
    )
    INSERT INTO raw.core_account (
        dt_transaction,
        dt_month,
        surrogate_key,
        cd_seqlan,
        ds_transaction_type,
        vl_transaction,
        id_transaction
    )
    SELECT * FROM df
"""

create_raw_core_pix = """
    CREATE TABLE IF NOT EXISTS raw.core_pix (
        dt_transaction          DATE        NULL,
        dt_month                INTEGER     NULL,
        cd_seqlan               INTEGER     NULL,
        ds_transaction_type     VARCHAR     NULL,
        vl_transaction          DOUBLE      NULL,
        id_transaction          VARCHAR     NULL
    );
    
    WITH df AS (
        SELECT 
            *
        FROM read_csv_auto('{PATH_RAW}/core_pix.csv')
    )
    INSERT INTO raw.core_pix (
        dt_transaction,
        dt_month,
        cd_seqlan,
        ds_transaction_type,
        vl_transaction,
        id_transaction
    )
    SELECT * FROM df
"""

queries = [
    create_raw_costumer,
    create_raw_core_pix,
    create_raw_core_account,
]

for query in tqdm(queries):
    db.sql("USE postgres; "+query)

100%|██████████| 3/3 [00:01<00:00,  1.88it/s]
